In [1]:
import pandas as pd
from sqlalchemy import create_engine
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings("ignore")

In [2]:
def forecast_yoy_growth_auto(df, col, forecast_steps=12, seasonal=True):
    """
    최적 SARIMAX 파라미터를 자동 탐색하여 수출입 증가율 예측.
    월별 데이터가 72개월 미만일 경우 예측을 수행하지 않습니다.

    Parameters:
        df (pd.DataFrame): 'period' (datetime), 예측 대상 컬럼 포함
        col (str): 예측 컬럼 (예: 'expDlr_yoy')
        forecast_steps (int): 예측 기간 (월 기준 12, 분기 기준 4)
        seasonal (bool): 계절성 사용 여부
    """
    df = df[['period', col]].dropna().copy()
    df = df.set_index('period').sort_index()
    ts = df[col]

    # ✅ 데이터 길이 검사
    if forecast_steps > 4 and len(ts) < 72:
        print("⚠️ 월 데이터가 6년치(72개월) 미만입니다. 충분한 데이터 기간 기준에 미달합니다.")
        return None
    elif forecast_steps <= 4 and len(ts) < 16:
        print("⚠️ 분기 데이터가 4년치(16분기) 미만입니다. 충분한 데이터 기간 기준에 미달합니다.")
        return None

    # SARIMA 파라미터 범위
    p = d = q = range(0, 2)
    pdq = [(x, y, z) for x in p for y in d for z in q]
    seasonal_pdq = [(x, y, z) for x in p for y in d for z in q]
    seasonal_period = 12 if forecast_steps > 4 else 4

    best_model, best_order, best_seasonal_order = sarimax_grid_search(ts, pdq, seasonal_pdq, seasonal_period)

    # 예측
    future_index = pd.date_range(
        start=ts.index[-1] + pd.offsets.DateOffset(months=1 if seasonal_period == 12 else 3),
        periods=forecast_steps,
        freq='M' if seasonal_period == 12 else 'Q'
    )

    forecast = best_model.forecast(steps=forecast_steps)
    forecast.index = future_index

    # 시각화
    plt.figure(figsize=(12, 6))
    plt.plot(ts.index, ts, label='Actual')
    plt.plot(forecast.index, forecast, label='Forecast', linestyle='--', color='red')
    plt.title(f"{col} 예측 (Auto SARIMAX)")
    plt.ylabel('YoY Growth')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [3]:
def load_monthly_trade_data_from_db():
    # host = '192.168.0.230'
    host = 'hystox74.synology.me'
    port = 3307
    user = 'stox7412'
    password = 'Apt106503!~'
    database = 'investar'
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')
    query = "SELECT * FROM trade_data_monthly"
    return pd.read_sql(query, con=engine)


def forecast_yoy_growth_auto_with_ci(df, col, hs_code, forecast_steps=12):
    warnings.filterwarnings("ignore")
    df = df[df['root_hs_code'] == hs_code].copy()

      # ✅ period 컬럼 자동 대체
    if 'period' not in df.columns:
        if 'date' in df.columns:
            df = df.rename(columns={'date': 'period'})
        elif 'year_month' in df.columns:
            df = df.rename(columns={'year_month': 'period'})
        elif df.index.name == 'period':
            df = df.reset_index()
        else:
            print(f"❌ {hs_code} | 누락된 컬럼: ['period'] 대체 불가")
            return None, None

    required_cols = ['period', col, 'expDlr', 'impDlr']
    missing_cols = [c for c in required_cols if c not in df.columns]
    if missing_cols:
        print(f"❌ {hs_code} | 누락된 컬럼: {missing_cols}")
        return None, None

    df = df[required_cols].dropna()
    df['period'] = pd.to_datetime(df['period'])
    df = df.set_index('period').sort_index()

    ts = df[col].astype(float)
    if forecast_steps > 4 and len(ts) < 72:
        print(f"⚠️ {hs_code}: 월 데이터가 6년치 미만입니다.")
        return None, None

    best_aic = np.inf
    best_order = None
    best_seasonal_order = None
    best_model = None

    p = d = q = range(0, 2)
    P = D = Q = range(0, 2)
    m = 12

    for order in [(p_, d_, q_) for p_ in p for d_ in d for q_ in q]:
        for seasonal_order in [(P_, D_, Q_, m) for P_ in P for D_ in D for Q_ in Q]:
            try:
                model = SARIMAX(ts,
                                order=order,
                                seasonal_order=seasonal_order,
                                enforce_stationarity=False,
                                enforce_invertibility=False)
                result = model.fit(disp=False)
                if result.aic < best_aic:
                    best_aic = result.aic
                    best_order = order
                    best_seasonal_order = seasonal_order
                    best_model = result
            except:
                continue

    if best_model is None:
        print(f"❌ {hs_code}: SARIMAX 모델 최적화 실패")
        return None, None

    forecast_result = best_model.get_forecast(steps=forecast_steps)
    pred_mean = forecast_result.predicted_mean
    conf_int = forecast_result.conf_int()

    future_index = pd.date_range(start=ts.index[-1] + pd.DateOffset(months=1), periods=forecast_steps, freq='M')

    forecast_df = pd.DataFrame({
        col + '_forecast': pred_mean.values,
        col + '_lower': conf_int.iloc[:, 0].values,
        col + '_upper': conf_int.iloc[:, 1].values
    }, index=future_index)

    print(f"✅ {hs_code}: 최적 모델 order={best_order}, seasonal_order={best_seasonal_order}, AIC={best_aic:.2f}")
    return forecast_df, df


def build_forecasted_series(base_df, forecast_df, col_name):
    full_df = pd.concat([base_df[[col_name]], forecast_df[[col_name + '_forecast']]], axis=0)

    full_df[col_name + '_forecast'] = pd.to_numeric(full_df[col_name + '_forecast'], errors='coerce')
    full_df[col_name + '_forecast'] = full_df[col_name + '_forecast'].replace([np.inf, -np.inf], np.nan)

    full_df[col_name + '_estimated'] = full_df[col_name + '_forecast'].shift(1) * (1 + full_df[col_name + '_forecast'])
    return full_df


def run_forecasting_pipeline(hs_code_list, max_hs_count=3):
    export_m_with_yoy = load_monthly_trade_data_from_db()
    hs_code_list = hs_code_list[:max_hs_count]
    result_list = []

    for hs_code in hs_code_list:
        print(f"📈 {hs_code} 예측 중...")
        for target_col in ['expDlr_yoy', 'impDlr_yoy']:
            forecast_df, base_df = forecast_yoy_growth_auto_with_ci(export_m_with_yoy, target_col, hs_code)

            if forecast_df is None or forecast_df.empty:
                print(f"⚠️ 예측 실패 또는 결과 없음 | hs_code={hs_code}, col={target_col}")
                continue

            full_df = build_forecasted_series(base_df, forecast_df, target_col)
            forecast_only = forecast_df.copy()
            forecast_only['hs_code'] = hs_code
            forecast_only['target_col'] = target_col
            forecast_only['date'] = forecast_only.index
            forecast_only = forecast_only.reset_index(drop=True)

            result_list.append(forecast_only)

    if not result_list:
        print("❌ 모든 예측이 실패하여 결과가 없습니다.")
        return pd.DataFrame()

    final_df = pd.concat(result_list, ignore_index=True)
    return final_df

def merge_forecast_with_actuals(forecast_df, trade_data_df):
    import pandas as pd

    # 복사 및 컬럼명 통일
    forecast_df = forecast_df.copy()
    forecast_df = forecast_df.rename(columns={'hs_code': 'root_hs_code'})

    forecast_df['period'] = pd.to_datetime(forecast_df['period'])

    # 예측 컬럼 존재 여부 확인
    forecast_cols = ['expDlr_yoy_forecast', 'impDlr_yoy_forecast']
    present_forecast_cols = [col for col in forecast_cols if col in forecast_df.columns]

    if not present_forecast_cols:
        raise KeyError("❌ forecast_df에 'expDlr_yoy_forecast' 또는 'impDlr_yoy_forecast' 컬럼이 없습니다.")

    # 📌 1. 예측 데이터 요약 (pivot 필요 없음)
    forecast_pivot = forecast_df[['period', 'root_hs_code'] + present_forecast_cols].drop_duplicates()

    # 📌 2. 무역 데이터 처리
    trade_data_df = trade_data_df.copy()
    trade_data_df['period'] = pd.to_datetime(
        trade_data_df['new_year'].astype(str) + '-' + trade_data_df['new_month'].astype(str) + '-01'
    )

    filtered_trade_data = trade_data_df[['period', 'root_hs_code', 'expDlr_yoy', 'impDlr_yoy', 'expDlr', 'impDlr']]

    # 📌 3. 예측과 실적 결합
    combined_df = pd.merge(
        filtered_trade_data,
        forecast_pivot,
        on=['period', 'root_hs_code'],
        how='outer'
    ).sort_values(['root_hs_code', 'period']).reset_index(drop=True)

    # 📌 4. 최종 YoY 생성
    combined_df['final_expDlr_yoy'] = combined_df['expDlr_yoy']
    if 'expDlr_yoy_forecast' in combined_df.columns:
        combined_df.loc[combined_df['final_expDlr_yoy'].isna(), 'final_expDlr_yoy'] = combined_df['expDlr_yoy_forecast']

    combined_df['final_impDlr_yoy'] = combined_df['impDlr_yoy']
    if 'impDlr_yoy_forecast' in combined_df.columns:
        combined_df.loc[combined_df['final_impDlr_yoy'].isna(), 'final_impDlr_yoy'] = combined_df['impDlr_yoy_forecast']

    # 📌 5. 자료형 정리
    combined_df['final_expDlr_yoy'] = pd.to_numeric(combined_df['final_expDlr_yoy'], errors='coerce')
    combined_df['final_impDlr_yoy'] = pd.to_numeric(combined_df['final_impDlr_yoy'], errors='coerce')

    # 📌 6. 12개월 전 실적 추출
    combined_df['expDlr_lag12'] = combined_df.groupby('root_hs_code')['expDlr'].shift(12)
    combined_df['impDlr_lag12'] = combined_df.groupby('root_hs_code')['impDlr'].shift(12)

    # 📌 7. 12개월 후 예측값 계산
    combined_df['expDlr_forecast_12m'] = combined_df['expDlr_lag12'] * (1 + combined_df['final_expDlr_yoy'])
    combined_df['impDlr_forecast_12m'] = combined_df['impDlr_lag12'] * (1 + combined_df['final_impDlr_yoy'])

    return combined_df

def pivot_forecast_df(forecast_df):
    # 피벗하여 target_col 기준으로 예측값을 컬럼화
    forecast_df = forecast_df.rename(columns={'hs_code': 'root_hs_code'})
    forecast_df['period'] = pd.to_datetime(forecast_df['period'])

    forecast_pivot = forecast_df.pivot_table(index=['period', 'root_hs_code'],
                                             columns='target_col',
                                             values='yoy_forecast').reset_index()

    # 컬럼 이름 정리
    forecast_pivot.columns.name = None
    forecast_pivot = forecast_pivot.rename(columns={
        'expDlr_yoy': 'expDlr_yoy_forecast',
        'impDlr_yoy': 'impDlr_yoy_forecast'
    })
    return forecast_pivot

def upload_forecast_to_db(df, db_info, table_name='trade_forecast_by_month'):
    """
    예측 결과 데이터를 MySQL(MariaDB)에 업로드하는 함수 (중복 시 대체)

    Parameters:
    - df (pd.DataFrame): 업로드할 데이터프레임
    - db_info (dict): DB 연결 정보. keys = ['host', 'port', 'user', 'password', 'database']
    - table_name (str): 업로드할 테이블 이름
    """
    try:
        # ✅ SQLAlchemy 엔진 생성
        engine = create_engine(
            f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
        )
        conn = engine.raw_connection()
        cursor = conn.cursor()

        # ✅ 테이블 생성 (없을 경우)
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            period DATE,
            root_hs_code VARCHAR(20),
            final_expDlr_yoy FLOAT,
            expDlr_forecast_12m FLOAT,
            PRIMARY KEY (period, root_hs_code)
        );
        """
        cursor.execute(create_table_query)

        # ✅ NaN 및 NaT, inf → None 변환 (모든 타입 안전하게 처리)
        df = df.astype(object).where(pd.notnull(df), None)

        # ✅ REPLACE INTO
        insert_query = f"""
        REPLACE INTO {table_name} (period, root_hs_code, final_expDlr_yoy, expDlr_forecast_12m)
        VALUES (%s, %s, %s, %s)
        """
        data = df[['period', 'root_hs_code', 'final_expDlr_yoy', 'expDlr_forecast_12m']].values.tolist()
        cursor.executemany(insert_query, data)

        conn.commit()
        cursor.close()
        conn.close()

        print(f"✅ 데이터가 {table_name} 테이블에 성공적으로 업로드되었습니다. (중복 시 자동 갱신)")

    except Exception as e:
        print(f"❌ 업로드 실패: {e}")

In [4]:
trade_data_df =  load_monthly_trade_data_from_db()

# hs_cd_list = trade_data_df['root_hs_code'].unique().tolist()
hs_cd_list = ['854232']
forecast_by_month = run_forecasting_pipeline(hs_code_list=hs_cd_list, max_hs_count=len(hs_cd_list))

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'hystox74.synology.me' (timed out)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [22]:
trade_data_df[trade_data_df['root_hs_code'] == '854232'].tail(12)

,root_hs_code,new_year,new_quarter,new_month,balPayments,expDlr,impDlr,region,year_month,expDlr_yoy,impDlr_yoy
48808,854232,2024,2,5,4039343143,5714466095,1675122952,전국,2024-05-01,0.889404168048076,0.13850403967384062
48809,854232,2024,2,6,5533943589,7055050042,1521106453,전국,2024-06-01,0.7489819124548467,0.1854118028747822
48810,854232,2024,3,7,4165965445,5804491763,1638526318,전국,2024-07-01,0.9018399133481385,0.4005168537892949
48811,854232,2024,3,8,4355896714,6034933835,1679037121,전국,2024-08-01,0.6192220431597901,0.4832809460804437
48812,854232,2024,3,9,5369305976,6926348924,1557042948,전국,2024-09-01,0.6238338591825219,0.40637841714910805
48813,854232,2024,4,10,4486588889,6220656370,1734067481,전국,2024-10-01,0.6585918439683798,0.36745004594587405
48814,854232,2024,4,11,4602695325,6443621669,1840926344,전국,2024-11-01,0.5122735527851219,0.6366506258975317
48815,854232,2024,4,12,5625206052,7444169908,1818963856,전국,2024-12-01,0.4006450483649675,0.39298385792244783
48816,854232,2025,1,1,3410142992,4766192111,1356049119,전국,2025-01-01,0.050511313104259825,-0.015376937446084704
48817,854232,2025,1,2,3303553240,4725936715,1422383475,전국,2025-02-01,-0.08706481567204005,-0.1329578299772447


In [11]:
forecast_by_month['period'] =  forecast_by_month['date']

In [12]:
# 예측 실행 결과 (from run_forecasting_pipeline)
forecast_df = forecast_by_month

# 'expDlr_yoy_forecast' → 'yoy_forecast'로 정리
if 'expDlr_yoy_forecast' in forecast_df.columns:
    forecast_df = forecast_df.rename(columns={'expDlr_yoy_forecast': 'yoy_forecast'})
elif 'impDlr_yoy_forecast' in forecast_df.columns:
    forecast_df = forecast_df.rename(columns={'impDlr_yoy_forecast': 'yoy_forecast'})

# DB에서 데이터 불러오기
trade_data_df = load_monthly_trade_data_from_db()

# 🧩 Step 1: forecast_df 피벗
forecast_df_pivoted = pivot_forecast_df(forecast_df)

# 🧩 Step 2: merge 실행'
final_combined_df = merge_forecast_with_actuals(forecast_df_pivoted, trade_data_df)

# hs_code_list = ['854232', '854231']
# hs_code_list
forecast_result_df = final_combined_df[final_combined_df['root_hs_code'].isin(hs_cd_list)]
forecast_result_data = forecast_result_df[['period', 'root_hs_code', 'final_expDlr_yoy', 'expDlr_forecast_12m']]

In [13]:
db_info = {
    # 'host' : '192.168.0.230',
    'host' : 'hystox74.synology.me',
    'port' : 3307,
    'user' : 'stox7412',
    'password' : 'Apt106503!~',
    'database' : 'investar'
}

upload_forecast_to_db(forecast_result_data, db_info)

✅ 데이터가 trade_forecast_by_month 테이블에 성공적으로 업로드되었습니다. (중복 시 자동 갱신)


In [19]:
forecast_result_data[forecast_result_data['root_hs_code'] == '854232']

,period,root_hs_code,final_expDlr_yoy,expDlr_forecast_12m
90153,2007-01-01,854232,NaN,NaN
90154,2007-02-01,854232,NaN,NaN
90155,2007-03-01,854232,NaN,NaN
90156,2007-04-01,854232,NaN,NaN
90157,2007-05-01,854232,NaN,NaN
...,...,...,...,...
90380,2025-12-31,854232,-0.103563,6.673226e+09
90381,2026-01-31,854232,0.042921,4.970762e+09
90382,2026-02-28,854232,0.045941,4.943051e+09
90383,2026-03-31,854232,0.057529,7.079393e+09


In [17]:
forecast_by_month.tail(12)

,expDlr_yoy_forecast,expDlr_yoy_lower,expDlr_yoy_upper,hs_code,target_col,date,impDlr_yoy_forecast,impDlr_yoy_lower,impDlr_yoy_upper,period
11184,NaN,NaN,NaN,903141,impDlr_yoy,2025-05-31,-0.188990,-7.329021,6.951041,2025-05-31
11185,NaN,NaN,NaN,903141,impDlr_yoy,2025-06-30,-0.315262,-8.511709,7.881185,2025-06-30
11186,NaN,NaN,NaN,903141,impDlr_yoy,2025-07-31,-0.086996,-8.607744,8.433751,2025-07-31
11187,NaN,NaN,NaN,903141,impDlr_yoy,2025-08-31,-0.696890,-9.332265,7.938484,2025-08-31
11188,NaN,NaN,NaN,903141,impDlr_yoy,2025-09-30,-0.858857,-9.540459,7.822745,2025-09-30
11189,NaN,NaN,NaN,903141,impDlr_yoy,2025-10-31,-0.898969,-9.602278,7.804340,2025-10-31
11190,NaN,NaN,NaN,903141,impDlr_yoy,2025-11-30,-0.646873,-9.362221,8.068474,2025-11-30
11191,NaN,NaN,NaN,903141,impDlr_yoy,2025-12-31,-0.652430,-9.375633,8.070773,2025-12-31
11192,NaN,NaN,NaN,903141,impDlr_yoy,2026-01-31,1.699072,-7.030028,10.428171,2026-01-31
11193,NaN,NaN,NaN,903141,impDlr_yoy,2026-02-28,0.525139,-8.210229,9.260507,2026-02-28
